<h3>Imports</h3>

In [1]:
from ocpa.objects.log.importer.csv import factory as ocel_import_factory
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.visualization.log.variants import factory as variants_visualization_factory

<h3>Import Test OCEL File</h3>
OCEL file downloadable from `http://ocel-standard.org/`. However, for testing a small OCEL file containing two process executions is used.

In [51]:
filename = "OCEL_example.jsonocel"
ocel = ocel_import_factory.apply(filename)

<h3>Variant Statistics</h3>

In [55]:
print("Number of process executions: "+str(len(ocel.process_executions)))
print("Number of variants: "+str(len(ocel.variants)))
variant_layouting = variants_visualization_factory.apply(ocel)

Number of process executions: 2
Number of variants: 2


In [57]:
print(variant_layouting[ocel.variants[0]])

([['A', [[0, 0], [2]]], ['B', [[1, 1], [2]]], ['E', [[0, 1], [1, 0]]], ['C', [[2, 2], [1, 2, 0]]], ['D', [[3, 3], [2]]], ['F', [[3, 3], [1, 0]]], ['G', [[4, 4], [0]]]], {0: ('items', 'items_1'), 1: ('items', 'items_2'), 2: ('orders', 'orders_1')})


In [58]:
print(variant_layouting[ocel.variants[1]])

([['D', [[3, 3], [1]]], ['C', [[2, 2], [1, 0]]], ['A', [[0, 0], [1]]], ['B', [[1, 1], [1]]]], {0: ('items', 'items_1'), 1: ('orders', 'orders_1')})


In [159]:
def OptionalActivity(subvariant1, subvariant2):
    #Case 1 and 2: One subvariant is empty, then the other subvariant is entirely optional
    if not len(subvariant1):
        keys = list(range(0, len(subvariant2), 1))
        opt = ["Opt" for i in range(0, len(subvariant2))]
        subvariant2 = zip(subvariant2, opt)
        return dict((zip(keys, subvariant2)))
    if not len(subvariant2):
        keys = list(range(0, len(subvariant1), 1))
        opt = ["Opt" for i in range(0, len(subvariant1))]
        subvariant2 = dict(zip(subvariant1, opt))
        return (zip(keys, subvariant1))
    else:
        result = dict()
        intersection = list(set(subvariant1).intersection(subvariant2))
        print("Intersection: " + str(intersection))
        posInResult = 0
        start1 = 0
        start2 = 0
        index1 = 0
        index2 = 0
        for i in range(len(intersection)):
            print("Indices of substrings: " + str(start1) + " and " + str(start2))
            index1 = subvariant1.index(intersection[i])
            index2 = subvariant2.index(intersection[i])
            print("Indices of intersection: " + str(index1) + " and " + str(index2))
            preceeding1 = subvariant1[start1:index1]
            preceeding2 = subvariant2[start2:index2]
            print("Non-matching preceeding subprocess of variant 1: " + str(preceeding1))
            print("Non-matching preceeding subprocess of variant 2: " + str(preceeding2))
            if(preceeding1==[]):
                for i in range(start2,index2):
                    result[posInResult] = (subvariant2[i],"Opt")
                    posInResult+=1
                    print(posInResult)
                result[posInResult] = (subvariant2[index2],"NOpt")
                print(subvariant1[index1+1:])  
                print(subvariant2[index2+1:])  
            elif(preceeding2==[]):
                for i in range(start1,index1):
                    result[posInResult] = (subvariant1[i],"Opt")
                    posInResult+=1
                    print(posInResult)
                result[index1] = (subvariant1[index1],"NOpt")
                print(subvariant1[index1+1:])  
                print(subvariant2[index2+1:]) 
            start1 = index1+1
            start2 = index2+1
            print(result)
        return result
        

In [160]:
optionalVariant = OptionalActivity(["E","T","A","C"],["A","B","C"])



Intersection: ['A', 'C']
Indices of substrings: 0 and 0
Indices of intersection: 2 and 0
Non-matching preceeding subprocess of variant 1: ['E', 'T']
Non-matching preceeding subprocess of variant 2: []
1
2
['C']
['B', 'C']
{0: ('E', 'Opt'), 1: ('T', 'Opt'), 2: ('A', 'NOpt')}
Indices of substrings: 3 and 1
Indices of intersection: 3 and 2
Non-matching preceeding subprocess of variant 1: []
Non-matching preceeding subprocess of variant 2: ['B']
3
[]
[]
{0: ('E', 'Opt'), 1: ('T', 'Opt'), 2: ('C', 'NOpt')}
